In [1]:
import pandas
import os

In [2]:
cells_of_interest=[x.split() for x in """18317_G10	DarkRed
20040_B1	DarkRed
20044_A10	DarkRed
20042_D4	DarkRed
18255_E8	DarkRed
18254_D2	DarkRed
18252_B12	DarkRed
20042_D2	DarkRed
18314_D6	DarkRed
18313_C10	DarkRed
20047_D6	DarkRed
19917_D1	DarkRed
18312_B4	DarkRed
18315_E6	DarkRed
18255_E7	DarkRed
18316_F6	DarkRed
20036_D11	DarkRed
19915_B9	DarkRed
20040_B6	DarkRed
20038_F1	DarkRed
20034_B6	DarkRed
18263_A2	DarkRed
18252_B8	DarkRed
18255_E4	red
18256_F2	red
18255_E5	red
18252_B11	red
20048_E11	red
18311_A9	red
18255_E11	red
20044_A12	red
18251_A5	red
20039_A1	red
20040_B12	red
20044_A5	red
18312_B11	red
18317_G2	red
20049_F3	red
20048_E12	red
18252_B1	red
18311_A6	red
20043_E2	red
18315_E10	red
18275_F6	red
18316_F11	red
20026_A5	red
20026_A6	red
18270_A4	red
18316_F7	red
18312_B6	red
18274_E7	red
18254_D12	red
18251_A3	red
20040_B2	red
20041_C1	red
18312_B8	red
18257_G6	red
18271_B2	red
18274_E3	red
18255_E2	red
18270_A11	red
20031_F8	green
20026_A12	green
17327_A4	green
17327_A12	green
19909_D11	green
18048_G2	green
18048_G8	green
18044_C6	green
19907_B3	green
18042_A7	green
18043_B2	green
18044_C9	green
17329_C5	green
18046_E8	green
17329_C10	green
18267_E6	green
17332_F7	green
17333_G6	green
18260_C8	green
18046_E3	green
18088_G2	green
18042_A4	green
18049_H11	green
18044_C5	green
19914_A12	green
18048_G4	green
18267_E11	green
18264_B10	green
18262_E11	green
18262_E6	green
20031_F9	green
19917_D7	green
17332_F9	green
18264_B9	green
18265_C6	green
18268_F6	green
20032_G5	green
20029_D10	green
17331_E1	green
20038_F11	green
18262_E2	green
17333_G2	green
17334_H3	green
17330_D3	green
17328_B1	green
17329_C9	green
18258_A7	green
17327_A2	green
17329_C1	green
17328_B5	green
17329_C8	green
17328_B8	green
20026_A9	green
20026_A8	green""".split('\n')]
cells_of_interest = [x[0] for x in cells_of_interest]

In [3]:
def is_cell_present(cells_of_interest, path):
    for cell in cells_of_interest:
        if cell in path:
            return True
    return False

In [4]:
with open('merge_bw_all.sh') as instream:
    cell_ids = set()
    uniq_bw = {}
    all_bw = {}
    for line in instream:
        args = line.rstrip().split(' ')[3:]
        cluster = args[0]
        for pathname in args[1:]:
            path, name = os.path.split(pathname)
            cell_id = name[:name.find('mm10')-1]
            if cell_id in cells_of_interest:
                cell_ids.add(cell_id)
                if name.endswith('_all.bw'):
                    all_bw.setdefault(cluster, []).append(pathname)
                elif name.endswith('_uniq.bw'):
                    uniq_bw.setdefault(cluster, []).append(pathname)
                else:
                    print(pathname)
                


In [5]:
sum([len(uniq_bw[k]) for k in uniq_bw])

115

In [6]:
set(cells_of_interest).difference(cell_ids)

set()

In [7]:
cell_ids.difference(cells_of_interest)

set()

In [8]:
len(set(cell_ids))

115

In [9]:
target = 'brian_subset_201905282223'

In [10]:
with open('merge_bw_{target}.sh'.format(target=target), 'wt') as outstream:
    for group in [uniq_bw, all_bw]:
        for cluster_name in group:
            args = ['python3', '../merge_bw.py', '-o', cluster_name]
            args.extend(group[cluster_name])
            outstream.write(' '.join(args))
            outstream.write(os.linesep)

In [11]:
os.getcwd()

'/woldlab/loxcyc/home/diane/proj/C1_mouse_limb_combined'

In [12]:
os.path.join(target, target+'.hub.txt')

'brian_subset_201905282223/brian_subset_201905282223.hub.txt'

In [13]:
hub="""hub hub
shortLabel {target}
longLabel {target}
genomesFile {target}.genomes.txt
email diane@caltech.edu
""".format(target=target)

with open(os.path.join(target, target+'.hub.txt'), 'wt') as outstream:
    outstream.write(hub)

In [14]:
genome="""genome mm10
trackDb mm10/trackDb-{target}.txt
""".format(target=target)

with open(os.path.join(target, target+'.genomes.txt'), 'wt') as outstream:
    outstream.write(genome)

In [15]:
trackDb = """track composite
shortLabel bigwigs
longLabel bigwigs
type bigWig
visibility full
dimensions dimX=multiread
sortOrder multiread
subGroup1 view Views signal=signal
subGroup2 multiread multiread all=all_reads uniq=unique_only
compositeTrack on

    track signal
    shortLabel signal
    longLabel signal
    type bigWig
    visibility full
    parent composite on
    view signal

            track greenall
            bigDataUrl http://woldlab.caltech.edu/~diane/C1_mouse_limb_combined/{target}/green-mm10-M4-male_all.bw
            shortLabel greenall
            longLabel Muscle 1 all
            type bigWig
            visibility full
            color 0,255,0
            subGroups multiread=all view=signal
            parent signal on
    
            track greenuniq
            bigDataUrl http://woldlab.caltech.edu/~diane/C1_mouse_limb_combined/{target}/green-mm10-M4-male_uniq.bw
            shortLabel greenuniq
            longLabel Muscle 1 uniq
            type bigWig
            visibility full
            color 0,255,0
            subGroups multiread=uniq view=signal
            parent signal on

            track redall
            bigDataUrl http://woldlab.caltech.edu/~diane/C1_mouse_limb_combined/{target}/red-mm10-M4-male_all.bw
            shortLabel redall
            longLabel Muscle 2 all
            type bigWig
            visibility full
            color 255,51,51
            subGroups multiread=all view=signal
            parent signal on
    
            track reduniq
            bigDataUrl http://woldlab.caltech.edu/~diane/C1_mouse_limb_combined/{target}/red-mm10-M4-male_uniq.bw
            shortLabel reduniq
            longLabel Muscle 2 uniq
            type bigWig
            visibility full
            color 255,51,51
            subGroups multiread=uniq view=signal
            parent signal on
    

            track DarkRedall
            bigDataUrl http://woldlab.caltech.edu/~diane/C1_mouse_limb_combined/{target}/DarkRed-mm10-M4-male_all.bw
            shortLabel DarkRedall
            longLabel Muscle 3 all
            type bigWig
            visibility full
            color 191,0,0
            subGroups multiread=all view=signal
            parent signal on
    
            track DarkReduniq
            bigDataUrl http://woldlab.caltech.edu/~diane/C1_mouse_limb_combined/{target}/DarkRed-mm10-M4-male_uniq.bw
            shortLabel DarkReduniq
            longLabel Muscle 3 uniq
            type bigWig
            visibility full
            color 191,0,0
            subGroups multiread=uniq view=signal
            parent signal on
""".format(target=target)

with open(os.path.join(target, 'mm10', 'trackDb-' + target +'.txt'), 'wt') as outstream:
    outstream.write(trackDb)

In [16]:
cmd="hubCheck http://woldlab.caltech.edu/~diane/C1_mouse_limb_combined/{target}/{target}.hub.txt".format(target=target)
print(cmd)

hubCheck http://woldlab.caltech.edu/~diane/C1_mouse_limb_combined/brian_subset_201905282223/brian_subset_201905282223.hub.txt
